In [1]:
from nn_architectures import Model1
from utils import train_loop
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Loading and Preprocessing Data

In [2]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

cols_to_drop = ['PassengerId', 'Ticket', 'Name', 'Cabin']
X_train = train_data.drop(cols_to_drop, axis=1).iloc[:, 1:]
y_train = train_data['Survived']
X_test = test_data.drop(cols_to_drop, axis=1)
y_test = pd.read_csv('data/gender_submission.csv').drop('PassengerId', axis=1)

In [3]:
embarked_labels = X_train['Embarked'].unique().copy()
sex_labels = X_train['Sex'].unique().copy()

for x in [X_train, X_test]:
    x.dropna(inplace=True)
    x['Embarked'] = x['Embarked'].map({e: i for i, e in enumerate(embarked_labels)})
    x['Sex'] = x['Sex'].map({e: i for i, e in enumerate(sex_labels)})

y_train = y_train.loc[X_train.index]
y_test = y_test.loc[X_test.index]

In [4]:
X_train = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float32).unsqueeze(dim=1)
y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), shuffle=False)

# Training Models

In [5]:
model = Model1(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

In [6]:
train_timeline = train_loop(model, optimizer, criterion, 2000, train_loader)

Epoch 0: Batch Loss - 14.52362060546875
Epoch 1: Batch Loss - 14.202311515808105
Epoch 2: Batch Loss - 13.953044891357422
Epoch 3: Batch Loss - 14.408599853515625
Epoch 4: Batch Loss - 13.773398399353027
Epoch 5: Batch Loss - 13.62300968170166
Epoch 6: Batch Loss - 13.42650032043457
Epoch 7: Batch Loss - 14.053539276123047
Epoch 8: Batch Loss - 13.6093111038208
Epoch 9: Batch Loss - 12.902918815612793
Epoch 10: Batch Loss - 12.163032531738281
Epoch 11: Batch Loss - 11.497210502624512
Epoch 12: Batch Loss - 11.015568733215332
Epoch 13: Batch Loss - 11.600576400756836
Epoch 14: Batch Loss - 11.351425170898438
Epoch 15: Batch Loss - 10.061853408813477
Epoch 16: Batch Loss - 11.708419799804688
Epoch 17: Batch Loss - 10.835406303405762
Epoch 18: Batch Loss - 10.229917526245117
Epoch 19: Batch Loss - 10.472268104553223
Epoch 20: Batch Loss - 10.556169509887695
Epoch 21: Batch Loss - 9.962639808654785
Epoch 22: Batch Loss - 9.97377872467041
Epoch 23: Batch Loss - 10.245270729064941
Epoch 24: 

In [7]:
with torch.inference_mode():
    preds = model(X_test)
    loss = criterion(preds, y_test)
    acc_mask = (torch.round(preds) == y_test).flatten()

loss, torch.count_nonzero(acc_mask)/len(acc_mask)

(tensor(4.4350), tensor(0.7795))